In [15]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [30]:
import subprocess as sp
import numpy as np
import os
import PIL.Image as Image

FFMPEG_BIN = "ffmpeg"

In [17]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def randfloat(a, b):
    return rand.random() * (b - a) + a

def my_print(s):
    sys.stdout.write(s)
    sys.stdout.flush()

In [18]:
def get_pipe(path):
    command = [ FFMPEG_BIN,
            '-i', path,
            '-f', 'image2pipe',
            '-pix_fmt', 'rgb24',
            '-vcodec', 'rawvideo', '-']
    pipe = sp.Popen(command, stdout = sp.PIPE, bufsize=10**8)
    return pipe

def next_frame(pipe, frame_size):
    length = frame_size[0] * frame_size[1] * frame_size[2]
    raw_image = pipe.stdout.read(length)
    if not raw_image:
        return None
    image =  np.fromstring(raw_image, dtype='uint8')
    image = image.reshape(frame_size)
    pipe.stdout.flush()
    return image

In [ ]:
FRAME_SIZE = (720, 1280, 3) # reversed
OUTPUT_SIZE = (720, 405)
#fps = 30
freq = 10

# for f in ['001', '002', '003']:
for f in ['002', '003']:
    video_path = './video/' + f + '.mp4'
    output_dir = './frames/' + f
    create_dir(output_dir)
    pipe = get_pipe(video_path)
    i = 0
    while True:
        if i % 1000 == 0:
            my_print('\rCurrent frame: %i' % i)
        image = next_frame(pipe, FRAME_SIZE)
        if image is None:
            break
        if i % freq == 0:
            resized = Image.fromarray(image).resize(OUTPUT_SIZE)
            image_path = os.path.join(output_dir, str(i / freq) + '.png')
            resized.save(image_path, 'PNG')
        i += 1
    

Current frame: 35000